In [11]:
import pandas_datareader.data as web
import pandas as pd
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import pandas as pd
import talib as ta
import importlib
import os
import platform
import psutil
import numpy as np
import BacktestingStrategies.Optimization as opt

#BTCJPY2時間足
#http://nipper.work/btc/index.php?market=bitFlyer&coin=BTCJPY&periods=7200&after=1640995200&before=1665100800
data = pd.read_csv('btc.csv', names=("Date", "Open", "High", "Low", "Close"))
data["Date"] = pd.to_datetime(data.index)
data.set_index('Date', inplace = True)
data = data.astype("double")

initial = 10_000_000
fname = f"html/plot-"
df_returns = pd.DataFrame()

In [12]:
print(data)

                          Open       High        Low       Close
Date                                                            
2022-01-01 00:00:00  5359000.0  5316222.0  5325242.0  190.414749
2022-01-01 02:00:00  5407484.0  5323749.0  5388884.0  212.975996
2022-01-01 04:00:00  5402039.0  5380820.0  5391012.0   90.025930
2022-01-01 06:00:00  5471828.0  5368104.0  5430000.0  202.119379
2022-01-01 08:00:00  5442174.0  5401395.0  5434087.0   88.946301
...                        ...        ...        ...         ...
2022-10-05 22:00:00  2915654.0  2891547.0  2898364.0  175.347057
2022-10-06 00:00:00  2922090.0  2893374.0  2912383.0   69.862984
2022-10-06 02:00:00  2950000.0  2909969.0  2945261.0  212.950939
2022-10-06 04:00:00  2954618.0  2931490.0  2941585.0  102.436067
2022-10-06 06:00:00  2955660.0  2927715.0  2935521.0  116.951120

[3336 rows x 4 columns]


In [13]:
def execute_backtest(bt):
    output = bt.run()
    print(output)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

In [14]:
def execute_optimize(bt, opt_params):
    output = bt.optimize(**opt_params)
    print(output)
    display(output._trades)
    display(output._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")
    df_temp = pd.DataFrame(output).transpose().drop(["_equity_curve", "_trades"], axis=1)
    df_temp["_strategy"] = str(output._strategy) + "_optimize"
    global df_returns
    df_returns = pd.concat([df_returns, df_temp], ignore_index=True)

In [16]:
import BacktestingStrategies.Strategy_MacdCross as btst
importlib.reload(btst)
importlib.reload(opt)

# # バックテストを設定
# bt = Backtest(
#     data, # チャートデータ
#     btst.MACDCross, # 売買戦略
#     cash=initial, # 最初の所持金
#     commission=0.00495, # 取引手数料
#     margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
#     trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
#     exclusive_orders=True, #自動でポジションをクローズ(オープン)
# )
# execute_backtest(bt)

# #最適化
# opt_params = {
#     "MACDshort": range(10, 25, 5),\
#     "MACDlong": range(10, 75, 10),\
#     "MACDsignal": range(10, 15, 5),\
#     "MACDThreshold": range(-100, 100, 10),\
#     "maximize": opt.optim_func,\
#     "method": 'grid',\
#     "constraint": lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
#     "max_tries": 1000
# }
# execute_optimize(bt, opt_params)


# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btst.MACDCross_WithShortPosition, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
execute_backtest(bt)

#最適化
execute_optimize(bt, opt_params)

Start                     2022-01-01 00:00:00
End                       2022-10-06 06:00:00
Duration                    278 days 06:00:00
Exposure Time [%]                   70.713429
Equity Final [$]                9279709.54435
Equity Peak [$]               23388372.067113
Return [%]                          -7.202905
Buy & Hold Return [%]               -38.58085
Return (Ann.) [%]                   -9.316753
Volatility (Ann.) [%]                     inf
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -99.543025
Avg. Drawdown [%]                  -41.619736
Max. Drawdown Duration      232 days 08:00:00
Avg. Drawdown Duration       27 days 11:00:00
# Trades                                   90
Win Rate [%]                        46.666667
Best Trade [%]                      13.079423
Worst Trade [%]                    -12.413055
Avg. Trade [%]                    

c:\Python310\lib\site-packages\backtesting\_stats.py:114: RuntimeWarning: overflow encountered in double_scalars
  s.loc['Volatility (Ann.) [%]'] = np.sqrt((day_returns.var(ddof=int(bool(day_returns.shape))) + (1 + gmean_day_return)**2)**annual_trading_days - (1 + gmean_day_return)**(2*annual_trading_days)) * 100  # noqa: E501


,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,41,49,5.378846e+06,5383599.0,-4752.73980,-0.000884,2022-01-04 10:00:00,2022-01-05 02:00:00,0 days 16:00:00
1,-2,67,83,4.980850e+06,4863456.0,234788.28280,0.023569,2022-01-06 14:00:00,2022-01-07 22:00:00,1 days 08:00:00
2,2,94,122,4.804666e+06,4892026.0,174720.10000,0.018182,2022-01-08 20:00:00,2022-01-11 04:00:00,2 days 08:00:00
3,-2,131,167,4.951369e+06,4963844.0,-24950.40000,-0.002520,2022-01-11 22:00:00,2022-01-14 22:00:00,3 days 00:00:00
4,2,177,208,5.004139e+06,4869500.0,-269278.96090,-0.026906,2022-01-15 18:00:00,2022-01-18 08:00:00,2 days 14:00:00
...,...,...,...,...,...,...,...,...,...,...
85,-3,3152,3167,2.711308e+06,2724908.0,-40799.22060,-0.005016,2022-09-21 00:00:00,2022-09-22 06:00:00,1 days 06:00:00
86,3,3185,3223,2.713365e+06,2775975.0,187830.00000,0.023075,2022-09-23 18:00:00,2022-09-26 22:00:00,3 days 04:00:00
87,-3,3231,3237,2.928432e+06,2784852.0,430740.45000,0.049030,2022-09-27 14:00:00,2022-09-28 02:00:00,0 days 12:00:00
88,-3,3247,3311,2.834400e+06,2849355.0,-44865.22500,-0.005276,2022-09-28 22:00:00,2022-10-04 06:00:00,5 days 08:00:00


c:\Python310\lib\site-packages\backtesting\_stats.py:114: RuntimeWarning: overflow encountered in double_scalars
  s.loc['Volatility (Ann.) [%]'] = np.sqrt((day_returns.var(ddof=int(bool(day_returns.shape))) + (1 + gmean_day_return)**2)**annual_trading_days - (1 + gmean_day_return)**(2*annual_trading_days)) * 100  # noqa: E501
c:\Python310\lib\site-packages\backtesting\_stats.py:114: RuntimeWarning: overflow encountered in double_scalars
  s.loc['Volatility (Ann.) [%]'] = np.sqrt((day_returns.var(ddof=int(bool(day_returns.shape))) + (1 + gmean_day_return)**2)**annual_trading_days - (1 + gmean_day_return)**(2*annual_trading_days)) * 100  # noqa: E501
c:\Python310\lib\site-packages\backtesting\_stats.py:114: RuntimeWarning: overflow encountered in double_scalars
  s.loc['Volatility (Ann.) [%]'] = np.sqrt((day_returns.var(ddof=int(bool(day_returns.shape))) + (1 + gmean_day_return)**2)**annual_trading_days - (1 + gmean_day_return)**(2*annual_trading_days)) * 100  # noqa: E501
c:\Python310\

Start                     2022-01-01 00:00:00
End                       2022-10-06 06:00:00
Duration                    278 days 06:00:00
Exposure Time [%]                   55.815348
Equity Final [$]                 6563451.1725
Equity Peak [$]               19966603.176355
Return [%]                         -34.365488
Buy & Hold Return [%]               -38.58085
Return (Ann.) [%]                  -42.354631
Volatility (Ann.) [%]     209083682621648....
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -67.127853
Avg. Drawdown [%]                  -25.545125
Max. Drawdown Duration      272 days 06:00:00
Avg. Drawdown Duration       68 days 15:00:00
# Trades                                  120
Win Rate [%]                        44.166667
Best Trade [%]                      15.581703
Worst Trade [%]                     -9.860144
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,42,50,5.404539e+06,5399500.0,5039.44625,0.000932,2022-01-04 12:00:00,2022-01-05 04:00:00,0 days 16:00:00
1,-2,67,83,4.980850e+06,4863456.0,234788.28280,0.023569,2022-01-06 14:00:00,2022-01-07 22:00:00,1 days 08:00:00
2,-2,100,108,4.831783e+06,4914686.0,-165806.60810,-0.017158,2022-01-09 08:00:00,2022-01-10 00:00:00,0 days 16:00:00
3,-2,111,122,4.870973e+06,4892026.0,-42106.51960,-0.004322,2022-01-10 06:00:00,2022-01-11 04:00:00,0 days 22:00:00
4,-2,131,144,4.951369e+06,5057652.0,-212566.40000,-0.021465,2022-01-11 22:00:00,2022-01-13 00:00:00,1 days 02:00:00
...,...,...,...,...,...,...,...,...,...,...
115,-2,3167,3188,2.711420e+06,2778680.0,-134520.58920,-0.024806,2022-09-22 06:00:00,2022-09-24 00:00:00,1 days 18:00:00
116,-2,3191,3223,2.734847e+06,2775975.0,-82255.67480,-0.015038,2022-09-24 06:00:00,2022-09-26 22:00:00,2 days 16:00:00
117,-2,3237,3247,2.771067e+06,2848500.0,-154866.03480,-0.027943,2022-09-28 02:00:00,2022-09-28 22:00:00,0 days 20:00:00
118,-2,3260,3273,2.817508e+06,2821331.0,-7646.08760,-0.001357,2022-09-30 00:00:00,2022-10-01 02:00:00,1 days 02:00:00


<Strategy MACDCross_WithShortPosition(MACDshort=15,MACDlong=20,MACDsignal=10,MACDThreshold=-100)>